In [1]:
import jieba

import pandas as pd
import numpy as np
from collections import Counter 

# Read Data: Company

In [2]:
company = pd.read_csv('company.csv')
print(len(company))
company.head()

604


,id,vat_number,name,address,jobs_count,g0v_data
0,7b1fce4b-5635-4a87-bb97-fee6f89280b3,17604757,飛哥文理補習班,台北市中正區衡陽路51號4樓,70,"{""error""=>true, ""message""=>""Company not found""}"
1,26455c36-97e4-42b1-871f-c651e672ee7c,50627725,陽光隔熱紙,桃園區介壽路457號,3,"{""data""=>{""地址""=>""桃園市桃園區福林里介壽路457號1樓"", ""現況""=>""核..."
2,4a53a43b-4022-4417-b2b7-c65be6fe54ef,72916919,善食堂,台北市大安區復興南路二段336號1樓,5,"{""data""=>{""財政部""=>{}}}"
3,123b5f59-56a8-4b45-bd2d-ff795c2ac350,10267365,悠菓創藝工作坊,高雄市新興區復興二路326號4樓之8,30,"{""data""=>{""地址""=>""高雄市新興區復興2路326號4樓之8"", ""現況""=>""核..."
4,f292dbb1-c0c7-497a-a3ef-5436da215f4e,12228473,神腦國際企業股份有限公司,新北市新店區中正路531號2樓,26,"{""data""=>{""財政部""=>{""行業""=>[[""272112"", ""有線電話製造""]]..."


In [3]:
# 建立一個 id 和 公司名稱的字典
id2name = {}
for i in range(len(company)):
    id2name[company['id'][i]] = company['name'][i]

# Read Data: job describe

In [4]:
job = pd.read_csv('job.csv')
print(len(job))
job.head()

1000


,id,title,desc,working_period,working_date,working_location,pay_wage_method,fulfil,hourly_wage,daily_wage,...,number_needed,job_candidates_count,featured,working_address,working_lat,working_lng,company_id,apply_question,administrative_area_level_3,term
0,5ddf51ad-9079-46fb-9ea0-289edc8dd579,日本展日文銷售人員,日文程度N1-N3左右工作內容主要為：協助日本師傅賣場銷售結帳翻譯日文介紹喜歡接觸人群有工作...,17：00-21：00平日17：00-22：00假日,6/26-7/8,忠孝sogo,活動結束後,False,170.0,NaN,...,6.0,8,False,106台北市大安區忠孝東路四段45號,25.041870,121.544633,43e94743-1b18-4a11-a012-2418dac3afc0,配合百貨營業時間,大安區,short_term
1,0f4243d8-77a3-4c7b-958b-11ed3cd74173,5/2 徵工讀生,桃園全大運活動協助工作人員 搬運桌椅 清點數量 桌巾 及收帆布,下午16:30-晚上20:30,2018/5/2,桃園中央大學,當日現領,True,150.0,NaN,...,20.0,33,False,NaN,NaN,NaN,7caadc54-251c-4bac-b931-f907817b224d,NaN,NaN,short_term
2,db0b76c4-69ff-4242-983e-94954719f19f,徵假日門市工讀生,1.門市客人接待，協助客人找到合適的童鞋2.門市環境清潔整理3.倉庫整理4.臨時交辦事項,早上10:00~下午18:00,2018/5/5~2019/12/31,三重,每月五日,False,150.0,NaN,...,3.0,0,False,241台灣新北市三重區捷運路19巷4弄4號,25.056412,121.484991,bfabdfc2-e0dc-4485-a4ba-19394ce020a1,NaN,三重區,long_term
3,296f2767-5abd-4f5b-9240-e17f84f8c559,徵平/假日門市工讀生,1.門市客人接待，協助客人找到合適的童鞋2.門市環境清潔整理3.倉庫整理4.臨時交辦事項,早上12:00~下午20:00,2018/5/3~2019/12/31,北投,每月五日,False,145.0,NaN,...,3.0,1,False,112台灣台北市北投區中央北路二段290號,25.138631,121.490447,bfabdfc2-e0dc-4485-a4ba-19394ce020a1,是否可配合排班,北投區,long_term
4,e6e010a6-5e24-4eb3-b373-e56781dcf096,徵外場人員、內場人員,歡迎熱情的你/妳們加入外場人員 工作內容協助點餐、帶位、送餐、收桌內場人員工作內容協助備料、...,早上11:00～下午21:00兩段班 中間休息2小時,即日起,水湳市場附近,隔月10號,False,140.0,NaN,...,4.0,1,False,406台灣台中市北屯區華美西街二段428號,24.174225,120.667934,dba152d9-3f63-4184-89d6-8581130475ed,是否可配合排班,北屯區,long_term


In [5]:
# 建立一個 id 和 求職描述
job_describe={}
for i in range(len(job)):
    job_describe[job['id'][i]] = job['desc'][i]

# Classification

In [8]:
# 主要類別
ind_word = pd.read_csv('ind_word.csv')
ind_word.columns

Index(['排除', '餐飲服務', '旅館飯店', '零售批發', '電信通訊', '物流倉儲', '資訊科技', '醫藥生物', '科技工程',
       '消費性產業', '國際貿易', '汽車製造', '財務金融', '石油製造', '休閒旅遊', '工程製造', '人力仲介', '廣告行銷',
       '媒體', '食品製造', '清潔'],
      dtype='object')

# Preparing Company describe data (string)

In [12]:
# 準備去除的符號和文字
symbol = ['0', '1', '2', '3', '4', '5', '6', '7', '8','9',
          '一', '二', '三', '四', '五', '六', '七', '八', '九',
          'a', 'u', 'J', 'A', 'B', 'F', 'Z', 'E', 'H', 'r', 'i', 'l','t', 'n',
          'e', 'o' ,'m', 's', 'g', 'C', 'p', 'y', 'f', 'd',
          'day', 'year', 'month','G', 'N' ,'R' ,'L', 'x',
           '"', '{', '}', '=', '>',  '\\', ' ',  '[', ']', ',', '、', 
            '）', '（', ')', '(', '.', ',', '/' ]

In [13]:
# 建立一個字典： {公司id：  公司名稱/相關描述/應徵描述}

company_string = {} # input string

for i in range(len(company)):
    com_id = company['id'][i] # take company id
    string = company['g0v_data'][i]
    
    # 找出相關字串的位置(index no.)    
    f1_1 = string.find('行業')+2
    f1_2 = string.find('資本額')-1
    
    if string.find('所營事業資料') == -1:
        f2_1 = string.find('營業項目')+4
    else:
        f2_1 = string.find('所營事業資料')+6
    f2_2 = string.find('登記機關')-1
    
    
    try:
        describe = company['name'][i]+string[f1_1:f1_2] + string[f2_1:f2_2]+ job_describe[com_id]
    except:
        describe = company['name'][i]+string[f1_1:f1_2] + string[f2_1:f2_2]
    
    
    # 刪除相關字元
    for s in symbol:
        describe = describe.replace(s, '')
        
    company_string[com_id] = describe

# Using Jieba to split the word

In [14]:
# Jieba分詞 ＆ 紀錄出現次數
counter = Counter()
for discribe in company_string.values():
    seg = jieba.cut(discribe, cut_all=False)
    token = [word for word in seg]
    counter.update(token)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.558 seconds.
Prefix dict has been built succesfully.


In [15]:
print(counter)

Counter({'零售': 583, '有限公司': 462, 'I': 436, '其他': 399, '姓名': 371, '業': 357, '服務業': 297, '公司': 282, '核准': 282, '所': 259, '法人': 255, '代表': 253, '出資額': 251, '職稱': 249, '設立': 236, '股份': 233, '日期': 217, 'h': 210, '董事': 205, '飲料': 193, '食品': 168, '批發業': 164, '經營': 164, '分公司': 164, '名單': 152, '批發': 149, '序號': 142, '及': 139, '，': 137, '或': 135, '非': 133, '得': 129, '法令': 129, '禁止': 129, '限制': 129, '之業務': 126, '管理': 123, '製': 120, '資料': 118, '名稱': 116, '顧問': 113, '狀況': 111, '台北市': 110, '所在地': 105, '店': 95, '財政部': 94, '人力': 94, '機關': 93, '用品': 90, '餐館': 89, '登記': 89, '事業': 89, '營業': 88, '除許': 86, '可業務外': 86, '店業': 82, '元': 80, '所營': 79, '代表人': 76, '號樓': 76, '董監事': 76, '資本': 76, '總額': 76, '設備': 74, '軟體': 73, '經理人': 70, '１': 69, '餐館業': 68, '什貨': 65, '造': 64, '服務': 64, '發業': 63, '電子': 63, '後': 61, '變': 60, '更': 60, '最': 59, '廣告': 57, '之': 57, '電腦': 56, '０': 56, '政府': 54, '分類': 54, '資訊': 53, '顧問業': 53, '品': 53, '地址': 52, '工程': 51, '未分類': 50, '出版': 50, '長': 48, '企業': 47, '文教': 46, '監察人': 46, '未': 46, '

In [16]:
useless = ['台灣', '台北', '台北市', '北市', '桃園', '大安','中壢區',  '敦化北路', '敦北', '中山' 
           '中正路', '市桃園區','高雄', '高雄市', '前鎮區', '新莊', '帕里', '新北市', '莊區', 
           '新泰',  '前鎮', '新竹市','新竹','中正', '內湖', '成功', '中國','南京','中山',
           '北屯', '中市', '桃園市', '松山' ,'明湖', '東路', '湖區', '信義區','新北',
           
           '民意', '登記', '機關', '政府', '代表人',  '經理人', '名單', '董監事', '職稱', '董事', 
           '出資額', '代表', '法人', '序號', '監察人', '資本', '總額', '所營', '事業', 
           '其他', '及其', '未分類', '之業務', '日期', '設立', 'yeamoh', 'yemoh','dyyemoh',
            '除許', '核准', '除外', '有限公司', '業除','業國際','富邦','財產','類型',
           '不得', '法令', '禁止', '限制','可業務外','供應','公室','名單', 
            '骨拉', '玄惠',  '指導委員會', '發國產', '林明宏', '服務','內設',
          '突出', '喬荃', '一樓', '有限', '責任', '名稱',  '所在地', '市政府',
           '\u3000', '姓名', '\xa0', '業無','產品','冷熱','商品','組織',
           '股份', '處理', '相關', '發業',  '店業', '綜合', '蔬菜', '黎明', '藝珂', 
           '人事', '分公司', '資訊供', '行業', '資本額', '營業',  '財政部','行業',
           '資本額','營業','地址','人名','稱藝','負責人','姓氏','使用','統一','發票',
           '總機','構統','編號','狀況','路段','號樓','經理','玉芬','總本','公司', 
            '青年', '業受', '花卉', '場所',  '統發票','經濟部','現況','項目',
           '統編號', '號設立', '公開',  '富商',  '聚鍋', '衡陽', '分類', '業化',  
           '業布', '金螺絲', '螺帽', '鉚釘', '許可業務外','路號', '週邊', '不動', 
           '料及', '水產品', '中心', '北市', '地下', '資本額', '貴金屬','農林', '漁礦']

In [32]:
# 給詞一個編號 & 只出現一次的去掉 & 只有一個字的去掉
word2idx = {}
idx2word = {}
num_words = 0
for word, count in counter.items():
    if count <= 10 or len(word)==1:
        continue
    if word in useless:
        continue
        
    word2idx[word] = num_words
    idx2word[num_words] = word
    num_words += 1

In [33]:
print('vector length: ', len(word2idx))
print('word index:\n', word2idx)

vector length:  136
word index:
 {'文理': 0, '補習': 1, '汽車': 2, '零件': 3, '配備': 4, '批發業': 5, '建材': 6, '零售': 7, '安裝': 8, '工程': 9, '網路': 10, '服裝': 11, '配件': 12, '衣著': 13, '鞋帽': 14, '傘服': 15, '飾品': 16, '文教': 17, '樂器育樂': 18, '用品': 19, '店面': 20, '設計業': 21, '國際': 22, '企業': 23, '電腦': 24, '設備': 25, '管理': 26, '資訊': 27, '技術': 28, '顧問': 29, '資料': 30, '電子': 31, '休閒': 32, '運動': 33, '批發': 34, '經紀': 35, '餐館': 36, '冰果': 37, '飲店': 38, '智慧': 39, '權業': 40, '顧問業': 41, '問業': 42, '服務業': 43, '軟體': 44, '般廣告': 45, '廣告': 46, '傳單': 47, '分送': 48, '市場': 49, '研究': 50, '調查業': 51, '藝文': 52, '會議': 53, '展覽': 54, '工商': 55, '業圖': 56, '出版': 57, '經營': 58, '商行': 59, '餐廳': 60, '食品': 61, '什貨批': 62, '什貨': 63, '飲料': 64, '貿易業': 65, '餐館業': 66, '電視': 67, '玩具': 68, '娛樂': 69, '投資': 70, '科技股份': 71, '代理': 72, '仲介': 73, '雜誌': 74, '整合': 75, '行銷': 76, '傘服飾': 77, '品批': 78, '演藝': 79, '活動業': 80, '館業': 81, '表演': 82, '租賃業': 83, '計業': 84, '人力': 85, '派遣': 86, '電器': 87, '安裝業': 88, '短期': 89, '科技': 90, '材料': 91, '事務性': 92, '機器': 93, '服飾品': 94, '活動': 9

# 人工處理規則
* 將產業納入可以互相重複的關鍵字，每筆資料分詞後，其有出現在該產業關鍵字，那一類就加1，分數最高的就是那一類！！
* 依分類結果來人工調整關鍵字
* 調幾次以後應該可以分的還可以

In [97]:
# 關鍵字
industry = {
 '餐飲服務': ['茶之魔手', '餐館', '冰果', '飲店', '飲料', '烘焙', '食品', '小吃店','早餐','餐飲業', '咖啡','廚房','餐廳','料理','店面', '外場'],
 '旅館飯店': ['旅館', '飯店', '酒店', ],
 '零售批發': ['批發業', '零售', '食品', '什貨批', '什貨', '加工','品批', '商行','店面','建材','服裝', '配件', '衣著', '鞋帽', '傘服', '飾品'], 
 '電信通訊': ['資訊', '電信', '通訊'], 
 '物流倉儲': ['貨運', '司機', '送貨', '倉管', '物流', '倉儲'], 
 '資訊科技': ['網路', '軟體','電腦' ,'資料'],
 '醫藥生物': ['醫藥', '生物', '科技','科技股份'], 
 '科技工程': ['電子', '科技', '機器', '科技股份','電腦','設備', '智慧'],
 '消費性產業': ['日常用品', '菸酒', '用品', '便利商店','服裝', '配件', '衣著', '鞋帽', '傘服', '飾品'], 
 '國際貿易': ['貿易', '國際', '國際貿易'],  
 '財務金融': ['金融', '銀行', '投資', '投資顧', '投資顧問'], 
 '休閒旅遊': ['休閒','娛樂','開發','運動'], 
 '工程製造': ['承裝業', '業機械', '建築', '安裝業', '機器', '工程', '安裝', '設備','製造', '實業'], 
 '人力仲介': [ '人力', '派遣', '租售','不動產','租賃業','仲介'], 
 '廣告行銷': ['廣告', '行銷', '傳單', '整合','市場'],
 '媒體': ['出版', '雜誌', '媒體', '電視', '廣播'], 
 '食品製造': ['食品','製造', '加工'], 
 '清潔': ['清潔', '打掃', ],
 '補習班':['文理', '補習','文教']
}

In [98]:
indus2ind = {}
ind2indus = {}
i = 0
for key in industry.keys():
    indus2ind[key] = i
    ind2indus[i] = key
    i += 1
print(indus2ind)

{'餐飲服務': 0, '旅館飯店': 1, '零售批發': 2, '電信通訊': 3, '物流倉儲': 4, '資訊科技': 5, '醫藥生物': 6, '科技工程': 7, '消費性產業': 8, '國際貿易': 9, '財務金融': 10, '休閒旅遊': 11, '工程製造': 12, '人力仲介': 13, '廣告行銷': 14, '媒體': 15, '食品製造': 16, '清潔': 17, '補習班': 18}


In [95]:
BoW = {}
idx2name = {}

train_data = np.zeros((len(company), len(indus2ind)))

i = 0
for com_id, describe in company_string.items():
    seg = jieba.cut(describe, cut_all=False)
    token = [word for word in seg]
    
    vector = np.zeros(len(indus2ind))
    for w in token:
        for key in industry.keys():
            if w in industry[key]:
                vector[indus2ind[key]] += 1
            #else:
            #    vector[19] += 1
    BoW[id2name[com_id]] = vector
    train_data[i,:] = vector
    idx2name[i] = id2name[com_id]
    i += 1

In [96]:
for key, array in BoW.items():
    idx = np.argwhere(BoW[key] == np.max(BoW[key]))[0][0]
    if idx == 0.:
        print(key,'===> ', '無法分辨')
    else:
        print(key, '===> ', ind2indus[idx])
    

飛哥文理補習班 ===>  補習班
陽光隔熱紙 ===>  零售批發
善食堂 ===>  無法分辨
悠菓創藝工作坊 ===>  零售批發
神腦國際企業股份有限公司 ===>  國際貿易
鵬泰顧問有限公司 ===>  科技工程
金哪吒休閒農場 ===>  休閒旅遊
新竹儒林 ===>  無法分辨
康體適能 ===>  休閒旅遊
玟穎貿易有限公司 ===>  國際貿易
金田餐飲股份有限公司 ===>  無法分辨
甘蔗媽媽古亭店 ===>  無法分辨
趨勢民調(台中辦公室) ===>  資訊科技
北基國際(股)公司北基中港加油站 ===>  無法分辨
有限責任台中市第一理貨勞動合作社 ===>  無法分辨
鹿聞商行 ===>  無法分辨
賞霖創藝股份有限公司 ===>  媒體
3M_台灣明尼蘇達礦業製造股份有限公司 ===>  無法分辨
傳承創新文教有限公司 ===>  消費性產業
手工洗鞋坊 ===>  無法分辨
黃昊數理補習班 ===>  資訊科技
遊戲橘子數位科技股份有限公司 ===>  媒體
明泓補習班 ===>  補習班
星樂整合行銷有限公司 ===>  零售批發
世雅育樂有限公司 ===>  零售批發
私立必揚英語短期補習班 ===>  補習班
展鵬科技有限公司 ===>  零售批發
詹浩數學補習班 ===>  無法分辨
綺非綠豆飲品 ===>  無法分辨
沐泳國際股份有限公司 ===>  無法分辨
泊泉商行 ===>  無法分辨
傅壹補習班 ===>  補習班
蘋果劇團 ===>  無法分辨
嘉隆富商業有限公司 ===>  消費性產業
凜洋茶飲行 ===>  無法分辨
亞典企業社 ===>  人力仲介
森學堂文理短期補習班 ===>  補習班
食樂企業社 ===>  無法分辨
私立黃昊文理美語短期補習班 ===>  補習班
圭信企業股份有限公司 ===>  無法分辨
盛合國際開發有限公司 ===>  人力仲介
萬星文化有限公司 ===>  人力仲介
艾沃克國際股份有限公司 ===>  零售批發
晶華國際酒店股份有限公司台南分公司 ===>  旅館飯店
元盛人力資源有限公司 ===>  人力仲介
宏儒補習班 ===>  補習班
數位小兔 ===>  零售批發
九湯屋日本拉麵樹林店 ===>  無法分辨
奕戰商行 ===>  消費性產業
曾理文理短期補習班 ==